In [1]:
from write_to_tf import * 
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

def generate_date_list(start, end):
    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')
    
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(convert_to_day_of_year(current_date.strftime('%Y%m%d')))
        current_date += timedelta(days=1)
    return date_list

total cores: 256
1 Physical GPUs, 1 Logical GPUs


2023-10-23 11:50:06.722407: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15744 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [2]:
folder = "/scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/"
bands = [29]
patch_size = 64
start = "20201001"
end = "20210430"
# start = "20210401"
# end = "20210430"
start = "20201201"
end = "20210430"
dates = generate_date_list(start, end)
start = "20211001"
end = "20211130"
dates.extend(generate_date_list(start, end))


# start_converted = convert_to_day_of_year(start)
# end_converted = convert_to_day_of_year(end)
# print(start_converted)
# print(end_converted)
x, dates, masks = extract_1km_data(folder, bands=bands, date_list=dates)
x, dates, masks = zip(*[(xi, date, mask) for xi, date, mask in zip(x, dates, masks) if (xi.shape[0] > 64) and (xi.shape[1] > 64)])
x = list(x)
dates = list(dates)


['/scratch/fslippe/modis/MOD02/daytime_1km/', '/scratch/fslippe/modis/MOD02/boundary_1km/', '/scratch/fslippe/modis/MOD02/night_1km/', '/scratch/fslippe/modis/MOD02/may-nov_2021/']


100%|██████████| 198/198 [03:42<00:00,  1.12s/it] 


In [5]:
max_value = float('-inf')
for x_s in x:
    current_max = np.max(x_s)
    if current_max > max_value:
        max_value = current_max

print(max_value)
np.save("/uio/hume/student-u37/fslippe/data/models/max_vals_dnb_l90_z50_(29)_%s-%s" %("cao_months_202012", "202111"), max_value)

15.703261


In [55]:
autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
all_patches = []
i=0
tot = len(x)
for (image, mask) in zip(x, masks):
    print("%s/%s" %(i, tot))
    patches, idx, n_patches = autoencoder.extract_patches(image, mask[-image.shape[0]:], mask_threshold=0.9)  # Assuming this function extracts and reshapes patches for a single image
    all_patches.append(patches)
    i+=1

patches = np.concatenate(all_patches, axis=0)

0/4401
1/4401
2/4401
3/4401
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/tensorflow/python/ops/control_flow_ops.py", line 2838, in while_loop
    return result  File "/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/tensorflow/python/ops/control_flow_ops.py", line 2786, in <lambda>
    body = lambda i, lv: (i + 1, orig_body(*lv))  File "/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/tensorflow/python/ops/map_fn.py", line 497, in compute
    return (i + 1, tas)  File "/opt/software/easybuild/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/tensorflow/python/op

In [56]:
len(patches)

669613

In [62]:
# TRAIN TEST SPLIT
patches, val_data = train_test_split(patches, test_size=0.15, shuffle=True)

In [65]:
###  NORMALIZE PATCHES 
model_run_name = "dnb_l90_z50_(29)_%s-%s" %("cao_months_202012", "202111")
print(model_run_name)
max_val = np.max(patches)
patches = patches / max_val
val_data = val_data / max_val

dnb_l90_z50_(29)_cao_months_202012-202111


In [68]:
patches_per_file = 250000 

for i in range(0, len(patches), patches_per_file):
    print(i)
    chunk = patches[i: i+patches_per_file]
    write_tfrecord(f'/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_trainingpatches_{model_run_name}_{i//patches_per_file}.tfrecord', chunk)

np.save("/uio/hume/student-u37/fslippe/data/models/max_vals_%s.npy" %(model_run_name), max_val)
np.save("/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_valpatches_%s" %(model_run_name), val_data)

0
250000
